In [90]:
# Ignore all levels of warnings and info which can be exist from Tensorflow
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import logging
logging.getLogger("tensorflow").setLevel(logging.WARNING)

In [2]:
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import re
import time
import pickle
pd.set_option('display.max_colwidth', 200)

In [3]:
import tensorflow as tf

In [4]:
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

Using TensorFlow backend.


In [5]:
tf.__version__

'1.15.2'

In [6]:
# read data
train = pd.read_csv("train_2kmZucJ.csv")
test = pd.read_csv("test_oJQbWVk.csv")

train.shape, test.shape

((7920, 3), (1953, 2))

In [7]:
train['label'].value_counts(normalize = True)

0    0.744192
1    0.255808
Name: label, dtype: float64

In [8]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful #cute #health #igers #iphoneonly #iphonesia #iphone
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xperia #S #sonyexperias… http://instagram.com/p/YGEt5JC6JM/
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smartphone #wifi #connect... http://fb.me/6N3LsUpCu
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #home http://instagr.am/p/Li_5_ujS4k/
4,5,1,What amazing service! Apple won't even talk to me about a question I have unless I pay them $19.95 for their stupid support!


In [9]:
# remove URL's from train and test
train['clean_tweet'] = train['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

test['clean_tweet'] = test['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [10]:
# remove punctuation marks
punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'

train['clean_tweet'] = train['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

# convert text to lowercase
train['clean_tweet'] = train['clean_tweet'].str.lower()
test['clean_tweet'] = test['clean_tweet'].str.lower()

# remove numbers
train['clean_tweet'] = train['clean_tweet'].str.replace("[0-9]", " ")
test['clean_tweet'] = test['clean_tweet'].str.replace("[0-9]", " ")

# remove whitespaces
train['clean_tweet'] = train['clean_tweet'].apply(lambda x:' '.join(x.split()))
test['clean_tweet'] = test['clean_tweet'].apply(lambda x: ' '.join(x.split()))

In [14]:
train.sample(10)

,id,label,tweet,clean_tweet
5607,5608,0,Photo: finally reached my bbs!!! #timberlands #timber #africaboots #boots #porfin #iphone #wuuuoopp http://tmblr.co/Zlnanu1agYATr,photo finally reached my bbs timberlands timber africaboots boots porfin iphone wuuuoopp
851,852,0,Photo: #quotes #heart #world #thinkpink #boys #girls #world #pictoftheday #photogram #iphone... http://tmblr.co/ZV-ZsufaK3am,photo quotes heart world thinkpink boys girls world pictoftheday photogram iphone...
1092,1093,0,My first app was just released in the App Store: http://bit.ly/socialcalculator #iphone #app,my first app was just released in the app store iphone app
7244,7245,1,When your headphones just aren't loud enough #firstworldproblems #deaf apple :(,when your headphones just aren't loud enough firstworldproblems deaf apple
7497,7498,0,"I’m way too blessed to be stressed and so are you! Don’t count the days, make the days count. Happy Thursday everyone! : @villagepsychicmusic #tbt #blessed #thursday #vermont #lakememphremagog #la...","i’m way too blessed to be stressed and so are you don’t count the days, make the days count. happy thursday everyone villagepsychicmusic tbt blessed thursday vermont lakememphremagog lakehouse sho..."
3931,3932,1,Fuck you Apple! You got rid of the camera roll and photo stream in the photos app? Wtf?! #Asshole #Apple #iOS8,fuck you apple you got rid of the camera roll and photo stream in the photos app wtf asshole apple ios
7385,7386,0,View point #vocation #newyear #trip #thailand #khaoko #a6000 #ILCE6000 #sony #vsco #snapspeed… http://instagram.com/p/xdyzt_L6cv/,view point vocation newyear trip thailand khaoko a ilce sony vsco snapspeed…
614,615,0,My favs #family #life #familytime #insta #instamood #iphone #snapshot #moment #picoftheday #be http://instagr.am/p/R5yerDqukd/,my favs family life familytime insta instamood iphone snapshot moment picoftheday be
5257,5258,1,I put in my password.. Why do I need to answer security questions?!?,i put in my password.. why do i need to answer security questions
5971,5972,0,"Me, Mom and Emily all got the same new phones on Dads new plan with sprint. It's called a motor 5 Google phone. Yay!!!! #sprint #motorola #phones https://www.instagram.com/p/Bl1mPh0gYTQ/?utm_sourc...","me, mom and emily all got the same new phones on dads new plan with sprint. it's called a motor google phone. yay sprint motorola phones …"


In [15]:
url = "https://tfhub.dev/google/elmo/2"
elmo = hub.Module(url)

In [89]:
# just a random sentence
x = ["Roasxxx ants are a popular snack in Columbia"]
x2 = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings = elmo(x, signature="default", as_dict=True)["elmo"]
embeddings2 = elmo(x2, signature="default", as_dict=True)["elmo"]

embeddings.shape

TensorShape([Dimension(1), Dimension(8), Dimension(1024)])

In [19]:
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [20]:
list_train = [train[i:i+100] for i in range(0,train.shape[0],100)]
list_test = [test[i:i+100] for i in range(0,test.shape[0],100)]

In [21]:
# Extract ELMo embeddings
elmo_train = [elmo_vectors(x['clean_tweet']) for x in list_train]
elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [22]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [34]:
elmo_train_new.shape

(7920, 1024)

In [23]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

In [26]:
# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [27]:
# load elmo_train_new
pickle_in = open("elmo_train_03032019.pickle", "rb")
elmo_train_new = pickle.load(pickle_in)

In [28]:
# load elmo_train_new
pickle_in = open("elmo_test_03032019.pickle", "rb")
elmo_test_new = pickle.load(pickle_in)

In [70]:
from sklearn.model_selection import train_test_split

xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  train['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

In [53]:
xtrain.shape

(6336, 1024)

In [54]:
ytrain.shape

(6336,)

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

lreg = LogisticRegression()
lreg.fit(xtrain, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [31]:
preds_valid = lreg.predict(xvalid)

In [32]:
f1_score(yvalid, preds_valid)

0.7831325301204819

In [33]:
# make predictions on test set
preds_test = lreg.predict(elmo_test_new)

In [71]:
from keras.utils import to_categorical
ytrain = to_categorical(ytrain)
yvalid = to_categorical(yvalid)

In [72]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

In [91]:
"""
# To feed raw string text into network, without extracting elmo features of text previously

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
"""

'\n# To feed raw string text into network, without extracting elmo features of text previously\n\ndef ELMoEmbedding(x):\n    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]\n\ninput_text = Input(shape=(1,), dtype=tf.string)\nembedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)\ndense = Dense(256, activation=\'relu\')(embedding)\npred = Dense(2, activation=\'softmax\')(dense)\nmodel = Model(inputs=[input_text], outputs=pred)\nmodel.compile(loss=\'categorical_crossentropy\', optimizer=\'adam\', metrics=[\'accuracy\'])\n'

In [73]:
input_embedding = Input(shape=(1024,))
dense = Dense(512, activation='relu')(input_embedding)
dense = Dense(128, activation='relu')(dense)
pred = Dense(2, activation='softmax')(dense)

model = Model(inputs=input_embedding, outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 1024)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_16 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 258       
Total params: 590,722
Trainable params: 590,722
Non-trainable params: 0
_________________________________________________________________


In [75]:
ytrain[0]

array([1., 0.], dtype=float32)

In [76]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(xtrain, ytrain, epochs=1, batch_size=32, validation_data=(xvalid,yvalid))
    model.save_weights('./elmo-model.h5')

Train on 6336 samples, validate on 1584 samples
Epoch 1/1
6336/6336 [==============================] - ETA: 1:32:03 - loss: 0.6623 - acc: 0.75 - ETA: 30:26 - loss: 0.6168 - acc: 0.7083 - ETA: 18:06 - loss: 0.5317 - acc: 0.73 - ETA: 12:49 - loss: 0.4979 - acc: 0.76 - ETA: 9:53 - loss: 0.4601 - acc: 0.7917 - ETA: 8:01 - loss: 0.4417 - acc: 0.804 - ETA: 6:43 - loss: 0.4379 - acc: 0.807 - ETA: 5:46 - loss: 0.4220 - acc: 0.818 - ETA: 5:02 - loss: 0.4058 - acc: 0.830 - ETA: 4:28 - loss: 0.3940 - acc: 0.835 - ETA: 4:00 - loss: 0.3772 - acc: 0.842 - ETA: 3:37 - loss: 0.3679 - acc: 0.847 - ETA: 3:18 - loss: 0.3582 - acc: 0.851 - ETA: 3:01 - loss: 0.3473 - acc: 0.855 - ETA: 2:47 - loss: 0.3610 - acc: 0.851 - ETA: 2:35 - loss: 0.3479 - acc: 0.855 - ETA: 2:24 - loss: 0.3464 - acc: 0.855 - ETA: 2:14 - loss: 0.3385 - acc: 0.859 - ETA: 2:06 - loss: 0.3365 - acc: 0.859 - ETA: 1:58 - loss: 0.3293 - acc: 0.863 - ETA: 1:51 - loss: 0.3233 - acc: 0.866 - ETA: 1:45 - loss: 0.3247 - acc: 0.866 - ETA: 1:39 - 

In [77]:
## Testing
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(xvalid, batch_size=32)

In [78]:
predicts = np.argmax(predicts, axis=1)
yvalid_back = np.argmax(yvalid, axis=1)

In [79]:
from sklearn import metrics

print(metrics.confusion_matrix(yvalid_back, predicts))

print(metrics.classification_report(yvalid_back, predicts))

[[1101   51]
 [ 153  279]]
             precision    recall  f1-score   support

          0       0.88      0.96      0.92      1152
          1       0.85      0.65      0.73       432

avg / total       0.87      0.87      0.87      1584

